In [1]:
import requests
from urllib.parse import quote
import base64
import time
from datetime import datetime

In [2]:
class TippiLink:
    def __init__(self, username, password, host="192.168.0.1"):
        self._username = username
        self._password = password
        self._host = host

    def _get_headers(self):
        headers = {
            'Referer': 'http://%s/userRpm/MenuRpm.htm' % self._host,
            'Authorization': self._auth_header(),
        }
        return headers

    def _auth_header(self):
        auth_bytes = bytes(self._username+':'+self._password, 'utf-8')
        auth_b64_bytes = base64.b64encode(auth_bytes)
        auth_b64_str = str(auth_b64_bytes, 'utf-8')

        auth_str = quote('Basic {}'.format(auth_b64_str))
        return auth_str
    
    def _get_mac_from_page(self, pageName):
        
        try:

            response = requests.get('http://%s/userRpm/%s.htm' % (self._host, pageName), headers=self._get_headers())
            content = response.content

            ini = 'hostList = new Array('
            fin = ');'

            texto = str(content)[str(content).index(ini)+len(ini)+2:]
            arrTx = texto[:texto.index(fin)].split("\"")
            arr = [l for l in arrTx if len(l.split('-')) == 6]

            return arr
        
        except:
            return "Error"
    
    def get_all_macs(self):
        page = '1'
        macs = []

        while True:
            arr = self._get_mac_from_page("WlanStationRpm.htm?Page="+page)
            page = str(int(page) + 1)

            if set(arr).issubset(set(macs)):
                break
            else:
                macs.extend(arr)

        mac_address = list(set(macs))

        mac_address = [mac.replace('-', ':').lower() for mac in mac_address]

        return mac_address
    
    def estadisticas(self):
        now = datetime.now()
        timestamp = datetime.timestamp(now)
        ini = 'wlanHostPara = new Array('
        ini1 = 'statistList = new Array('
        fin = ');'

        try:
            # numero de conectados
            response = requests.get('http://%s/userRpm/%s.htm' % (tl._host, 'WlanStationRpm.htm'), headers=tl._get_headers())
            content = response.content
            lista = str(content)[str(content).index(ini)+len(ini):str(content).index(fin)].replace('\\n', '').split(',')
            nconx = lista[5]

            # enviados y recibidos
            response = requests.get('http://%s/userRpm/%s.htm' % (tl._host, 'StatusRpm.htm'), headers=tl._get_headers())
            content = response.content
            texto = str(content)[str(content).index(ini1)+len(ini1)+2:]
            statistList = texto[:texto.index(fin)].split(',')

            b_rec = statistList[0]
            b_env = statistList[1]
            
            return [timestamp, nconx, b_rec, b_env]
        except:
            return "Error"
        
    def restart_router(self):
        params = (('Reboot', 'Reboot'),)
        try:
            res = requests.get('http://%s/userRpm/SysRebootRpm.htm' % self._host,
                                headers=self._get_headers(), params=params)
        except:
            pass
    

In [3]:
# https://pypi.org/project/python-dotenv/
from dotenv import load_dotenv
import os

load_dotenv()
tplink_id = os.getenv("TPLINK_ID")
tplink_pw = os.getenv("TPLINK_SECRET")

tl = TippiLink(tplink_id, tplink_pw, "192.168.66.32")


In [4]:
print(tl.estadisticas())

[1612705879.520507, ' 23', '151106479', ' 411486075']


In [5]:
tl.get_all_macs()

['20:64:32:7b:c8:cd',
 'e4:c8:01:c9:60:50',
 '80:5a:04:dc:32:4d',
 'a2:01:3d:d1:f2:33',
 '1c:56:fe:be:59:97',
 '8c:c5:e1:dd:73:83',
 '94:27:90:5a:8b:e8',
 '60:14:66:66:ef:71',
 'bc:44:86:90:fc:ca',
 '00:8a:0f:05:57:66',
 '00:91:10:71:66:95',
 'e0:cc:f8:38:16:9d',
 'dc:44:b6:c5:9c:cf',
 'b4:1c:30:fa:e2:c6',
 '60:14:66:72:bc:03',
 '0e:1a:38:98:5c:0b',
 '80:9b:20:9f:dc:58',
 'f8:1a:67:cf:ef:82',
 '00:91:10:74:01:74',
 '04:79:70:8a:fc:48',
 '44:6d:57:df:38:87',
 'bc:98:df:79:a3:75',
 '0c:2c:54:58:c3:6f',
 '80:9b:20:db:f5:c0',
 '90:f6:52:e1:95:4e',
 'd4:c9:4b:37:8e:fa',
 'b0:e0:3c:0f:11:42',
 'd4:72:26:0c:c2:83',
 '60:14:66:d0:6c:94',
 '86:35:f1:c4:37:f3',
 '9c:2a:83:75:e8:79']